# The goal of this script is to calculate metagenome read mapping levels to KEGG-annotated genes. Before running it, you will need to have extracted open reading frames (ORFs) with Prodigal or a similar program, run KofamScan on those ORFs, and mapped reads to your assembly contigs with Magic Blast. 

## To run KofamScan, see these instructions from Taylor Reiter: https://taylorreiter.github.io/2019-05-11-kofamscan/

## To map reads to your assembly with Magic Blast, see Roth Conrad's GitHub:
## https://github.com/rotheconrad/00_in-situ_GeneCoverage

# To run this script you will need the following:
## 1. "Master" KO text file ("KO_Orthology_ko00001.txt"). Instructions for generating this file or downloading it are here: http://merenlab.org/2018/01/17/importing-ghostkoala-annotations/
## 2. KofamScan output file (eg, "sample_kofams.txt")
## 3. Filtered Magic Blast output for unassembled reads run against the predicted gene sequences (ORFs). The file should end with "fltrdBestHits.blst". See Roth Conrad's Github for how to process Magic Blast output. 

## Be sure to change all the paths in this file to the correct ones for your system.

#### Import dependencies

In [1]:
import os as os
import glob as glob
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/npatin3/miniconda/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Navigate to working directory containing kofamscan output files
os.chdir('/Users/npatin3/Dropbox (GaTech)/OV/Full_timeseries/Metagenomes/kofamscan')

#### Import the "master" KO htext file

In [3]:
ko_master = '/Users/npatin3/Dropbox (GaTech)/Workflows/KO_Orthology_ko00001.txt'

# 1. Use the KofamScan annotations and Magic Blast read mapping results to calculate the number of reads mapped to each gene (KO)

### Function to create file with KOs and number of mapped reads to each KO

In [4]:
def get_KO_readcounts(kofams, magicblast, name):
    # Drop rows with no KO
    kofams.dropna(inplace=True)
    # Select query and reference ORF from magicblast output
    mb = magicblast[['Read','ORF']]
    # Merge KO numbers with ORFs in MagicBlast results
    mb_kos = mb.merge(kofams, on='ORF', how='inner')
    # Add 'Size' column to provide count of one for each row
    mb_kos['Size'] = 1
    # Group by KO and provide sum of reads mapped to each one. Show one result for each KO only.
    mb_kos['ReadCounts'] = mb_kos.groupby(['KO'])['Size'].transform('sum')
    mb_kos.drop_duplicates(subset=['KO'], inplace=True)
    KOs_readcounts = mb_kos[['KO','ReadCounts']]
    KOs_readcounts['Sample'] = name
    return(KOs_readcounts)

### Make a directory called 'kofamscan' that ONLY contains the KofamScan output text files (1 file per sample). This code assumes those files are named as follows: "X_Y_kofams.txt"

In [7]:
# Show all the KofamScan output files in the directory described above
# There should be one file per sample, each of which contains two columns: one for ORF number, one for associated KO 
# os.chdir('kofamscan')
for file in glob.glob("*kofams.txt"):
    a, b = file.split('.')
    c, d, e = a.split('_')
    name = c
    print(file, a, name)

## Get read counts for each KO

### This loop will match each kofamscan output with its associated sampe Magic Blast output, and merge all sample-specific files together in a dataframe showing the sample-specific KOs and read counts

In [ ]:
reads = []
for file in glob.glob("*kofams.txt"):
    kofams = pd.read_csv(file, sep='\t', names=['ORF', 'KO'])
    a, b = file.split('.')
    name, d = a.split('_')
    # Depending on how your Magic Blast output file is named may need to modify the file name
    mb = pd.read_csv('%s_magicblast.fltrdBstHts.blst' % name, comment='#', sep='\t', header=None)
    mb.columns = ["Read","ORF","% identity","not used","not used",
                      "not used","query start","query end","reference start",
                      "reference end","not used","not used","score",
                      "query strand","reference strand","query length",
                      "BTOP","num placements","not used","compartment",
                      "left overhang","right overhang","mate reference",
                      "mate ref. start","composite score"]
    mb = mb[['Read','ORF']]
    df = get_KO_readcounts(kofams, mb, name)
    reads.append(df)

reads_df = pd.concat(reads)

In [14]:
reads_df.head()

,KO,ReadCounts,Sample
0,K16554,2751,080516
2710,K07089,842,080516
3546,K07568,1137,080516
4666,K03217,1860,080516
6493,K01733,3208,080516


In [15]:
# Check size of the dataframe
reads_df.size

54456

In [16]:
# Export dataframe to csv
reads_df.to_csv("OV_assembly_KOs_readcounts.csv", index=None)

# 2. Now you can normalize the read counts by Genome Equivalent values. Similar to the KofamScan outputs, you should have a directory called 'GEs' or something similar that contains only the Microbe Census output files. As with the KofamScan output files, this assumes the file names are formatted as follows: "X_Y_GE.txt". 

In [10]:
# Import the csv file just created, if necessary
reads_df = pd.read_csv("OV_assembly_KOs_readcounts.txt", sep='\t', header=0, dtype='str')

#### Function to extract GE from indiivdual MicrobeCensus report

In [8]:
def extract_GE(GE):
    """Use the MicrobeCensus output file to extract the Genome Equivalent value of the metagenome"""
    lines = []
    with open(GE, "rt") as GE_file:
        for line in GE_file:
            lines.append(line)
    for s in lines[12].split():
        try:
            ge = float(s)
        except ValueError:
            pass
    return(ge)

#### Move to directory with GE files and run function through loop

In [9]:
os.chdir('/Users/npatin3/Dropbox (GaTech)/OV/Full_timeseries/Metagenomes/GEs')

In [10]:
GE_values = {}
files = os.listdir()
for file in files:
    a, b = file.split('.')
    c, d, e = a.split('_')
    x = extract_GE(file)
    GE_values[c] = x

In [11]:
GE_values

{'083018': 446.144900054,
 '080516': 751.835105984,
 '092816': 627.392761456,
 '101019': 1766.84507874,
 '042618': 814.232063266,
 '021419': 1620.28429227}

### Match KOs, samples, and GE values

In [12]:
# Map the GE of the corresponding sample onto the read counts file
reads_df['GE'] = reads_df['Sample'].map(GE_values)

In [13]:
reads_df.head()

,KO,ReadCounts,Sample,GE
0,K16554,2751,080516,751.835106
1,K07089,842,080516,751.835106
2,K07568,1137,080516,751.835106
3,K03217,1860,080516,751.835106
4,K01733,3208,080516,751.835106


### Make a dict of the KOs from the reads df

In [14]:
# Make a dict from the reads df
kolist = reads_df['KO']
ko_dict = {}

In [15]:
for line in kolist:
    konumber = line.rstrip()
    ko_dict[konumber] = ''

### Match the KO number in column 3 of the master file with the KOs using the ko_dict keys, and export this combined data frame as 'kolist_full.tsv'

In [16]:
os.chdir('/Users/npatin3/Dropbox (GaTech)/OV/Full_timeseries/Metagenomes/kofamscan')

In [17]:
with open(ko_master, 'r') as file, open('kolist_full.tsv', 'w') as outfile:
    for line in file:
        X = line.rstrip().split('\t')
        konumber = X[3].split(' ')[0]
        if konumber in ko_dict:
            outfile.write(line)

# 3. Merge the spreadsheet of KO annotations with the read counts file to get a full spreadsheet of genes and normalized read counts for each one.

In [36]:
# you can name these columns however you want, just make sure to adjust the function below accordingly
kolist_full = pd.read_csv('kolist_full.tsv', sep='\t', header=None, names=['One','Two','Three','Four']) 
kolist_full.head()

,One,Two,Three,Four
0,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K00845 glk; glucokinase [EC:2.7.1.2]
1,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],"K01810 GPI, pgi; glucose-6-phosphate isomeras..."
2,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K13810 tal-pgi; transaldolase / glucose-6-pho...
3,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K15916 pgi-pmi; glucose/mannose-6-phosphate i...
4,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],"K00850 pfkA, PFK; 6-phosphofructokinase 1 [EC..."


In [37]:
def reformat_kolist_full (ko_df, reads_df):
    """ First reformat the KO annotation file"""
    # Convert all column values to strings
    ko_df['One'] = ko_df['One'].astype(str)
    ko_df['Two'] = ko_df['Two'].astype(str)
    ko_df['Three'] = ko_df['Three'].astype(str)
    ko_df['Four'] = ko_df['Four'].astype(str)
    # Remove the numbered codes preceding each category
    ko_df[['One', 'Group']] = ko_df['One'].str.split(" ", n=1, expand=True)
    ko_df[['Two', 'Subgroup']] = ko_df['Two'].str.split(" ", n=1, expand=True)
    ko_df[['Three', 'Subgroup2']] = ko_df['Three'].str.split(" ", n=1, expand=True)
    ko_df[['Four', 'Function']] = ko_df['Four'].str.split(" ", n=1, expand=True)
    # Rename the column with KO numbers
    ko_df_new = ko_df.rename(columns={"Four": "KO"})
    # Select only relevant columns
    ko_df_new = ko_df_new[['Group','Subgroup','Subgroup2','KO','Function']]
    """ Now the KO annotation file can be merged with the read counts file"""
    # Merge this data frame with the read counts data frame
    df_full = ko_df_new.merge(reads_df, on='KO', how='inner')
    df_full['ReadCounts'] = df_full['ReadCounts'].astype(float)
    df_full['GE'] = df_full['GE'].astype(float)
    # Normalize read counts by GE value of metagenome
    df_full['ReadCounts_Norm'] = df_full['ReadCounts'] / df_full['GE']
    return(df_full)

In [38]:
KOs_readcounts_full = reformat_kolist_full(kolist_full, reads_df)
KOs_readcounts_full.head()

In [41]:
# Export this reformatted sheet
KOs_readcounts_full.to_csv('KO_functions_reads.csv', index=None)